<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/functionalcpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
1. C++で関数型プログラミング風に書きたい、と思った。
1. functional programming in C++ という本のchapter01がネットで読めるので、導入だけでもいいと思って読んでいるところ。
1. https://itbook.store/files/9781617293818/chapter1.pdf

# 関数型プログラミング入門

In [ ]:
# 最初にでてくる、関数型でない書き方の例を、とりあえず動かす。
%%writefile countlines01.cpp
#include <iostream>
#include <fstream>
#include <vector>
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        std::ifstream in(file);
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}
int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines01.cpp


In [ ]:
!g++ countlines01.cpp -o countlines01; ./countlines01

29


In [ ]:
!wc countlines01.cpp

 29  75 702 countlines01.cpp


行数が一致することを確認。

In [ ]:
# std::count アルゴリズムを使う
%%writefile countlines02.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    for (const auto& file : files) {
        results.push_back(count_lines(file));
    }
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines02.cpp


In [ ]:
!g++ countlines02.cpp -o countlines02; ./countlines02

29


In [ ]:
%%writefile countlines03.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results(files.size());
    std::transform(files.cbegin(), files.cend(), results.begin(), count_lines);
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines03.cpp


In [ ]:
!g++ countlines03.cpp -o countlines03; ./countlines03

29


In [40]:
# 次のコードはRangesを使っていて、いまの段階では動かない。
%%script false
%%writefile countlines04.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    return files | transform(count_lines);
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


In [ ]:
# 次のコードはRangesを使っていて、いまの段階では動かない。
%%script false
%%writefile countlines05.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    return files | transform(open_file) | transform(count_lines);
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


In [ ]:
!g++ countlines05.cpp -o countlines05; ./countlines05

In [41]:
#実験
%%writefile transform01.cpp
#include <algorithm>
#include <cctype>
#include <functional>
#include <iostream>
#include <string>
#include <vector>

int main() {
    std::string s("hello");

    namespace ranges = std::ranges;

    ranges::transform(
        s.begin(), s.end(), s.begin(),
        [](unsigned char c) -> unsigned char { return std::toupper(c); });

    std::vector<std::size_t> ordinals;
    ranges::transform(s, std::back_inserter(ordinals),
                      [](unsigned char c) -> std::size_t { return c; });

    std::cout << s << ':';
    for (auto ord : ordinals) {
        std::cout << ' ' << ord;
    }

    ranges::transform(ordinals, ordinals, ordinals.begin(), std::plus{});

    std::cout << '\n';
    for (auto ord : ordinals) {
        std::cout << ord << ' ';
    }
    std::cout << '\n';
}

Writing transform01.cpp


In [45]:
!g++  -std=gnu++17 transform01.cpp -o transform01; ./transform01

transform01.cpp: In function ‘int main()’:
transform01.cpp:11:29: error: ‘ranges’ is not a namespace-name
     namespace ranges = std::ranges;
                             ^~~~~~
transform01.cpp:11:35: error: expected namespace-name before ‘;’ token
     namespace ranges = std::ranges;
                                   ^
transform01.cpp:13:5: error: ‘ranges’ has not been declared
     ranges::transform(
     ^~~~~~
transform01.cpp:18:5: error: ‘ranges’ has not been declared
     ranges::transform(s, std::back_inserter(ordinals),
     ^~~~~~
transform01.cpp:26:5: error: ‘ranges’ has not been declared
     ranges::transform(ordinals, ordinals, ordinals.begin(), std::plus{});
     ^~~~~~
/bin/bash: ./transform01: No such file or directory


In [99]:
%%writefile range01.cpp
#include <iostream>
#include <vector>
#include <functional>
#include <range-v3>

int main (){
    return 0;
}

Overwriting range01.cpp


In [100]:
!g++   range01.cpp -o range01; ./range01

range01.cpp:4:10: fatal error: range-v3: No such file or directory
 #include <range-v3>
          ^~~~~~~~~~
compilation terminated.
/bin/bash: ./range01: No such file or directory


In [71]:
!git clone https://github.com/Microsoft/vcpkg.git
!cd vcpkg/


fatal: destination path 'vcpkg' already exists and is not an empty directory.


In [84]:
!ls ./vcpkg

bootstrap-vcpkg.bat  docs	  ports		   README.md	    triplets
bootstrap-vcpkg.sh   format.diff  README_es.md	   README_zh_CN.md
CHANGELOG.md	     LICENSE.txt  README_fr.md	   scripts
CONTRIBUTING.md      NOTICE.txt   README_ko_KR.md  toolsrc


In [85]:
!./vcpkg/bootstrap-vcpkg.sh
!./vcpkg integrate install
!./vcpkg install range-v3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   652  100   652    0     0   4100      0 --:--:-- --:--:-- --:--:--  4100
100 39.8M  100 39.8M    0     0  47.5M      0 --:--:-- --:--:-- --:--:-- 47.5M
Extracting cmake...
Extracting cmake... done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   637  100   637    0     0   3956      0 --:--:-- --:--:-- --:--:--  3956
100 99913  100 99913    0     0   348k      0 --:--:-- --:--:-- --:--:--  348k
Extracting ninja...
Extracting ninja... done.
-- The CXX compiler identification is GNU 7.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++-7 - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting the

In [91]:
!cd vcpkg; ./bootstrap-vcpkg.sh

-- The CXX compiler identification is GNU 7.5.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++-7 - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting the C++ compiler in use
-- Detecting the C++ compiler in use - gcc
-- Detecting the C++ standard library
-- Detecting the C++ standard library - libstdc++
-- Detecting how to use the C++ filesystem library
-- Detecting how to use the C++ filesystem library - <experimental/filesystem> with -lstdc++fs
-- Looking for C++ include pthread.h
-- Looking for C++ include pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Check if compiler accepts -pthread
-- Check if compiler accepts -pthread - yes
-- Found Threads: TRUE  
-- Configuring done
-- Generating done
-- Build files have been written to: /content/vcpkg/toolsrc/build.rel
[0/2] Re-checking globbed directo

In [94]:
!cd vcpkg; ./vcpkg integrate install; ./vcpkg install range-v3

Applied user-wide integration for this vcpkg root.

CMake projects should use: "-DCMAKE_TOOLCHAIN_FILE=/content/vcpkg/scripts/buildsystems/vcpkg.cmake"
Computing installation plan...
The following packages will be built and installed:
    range-v3[core]:x64-linux -> 0.11.0
Detecting compiler hash for triplet x64-linux...
Could not locate cached archive: /root/.cache/vcpkg/archives/db/db9a366519016f7e7b920b512096571b434b3832.zip
Starting package 1/1: range-v3:x64-linux
Building package range-v3[core]:x64-linux...
-- Downloading https://github.com/ericniebler/range-v3/archive/4989f3e9ff2efee1852942bb9328ef121369ba02.tar.gz...
-- Extracting source /content/vcpkg/downloads/ericniebler-range-v3-4989f3e9ff2efee1852942bb9328ef121369ba02.tar.gz
-- Using source at /content/vcpkg/buildtrees/range-v3/src/121369ba02-823af3dba2.clean
-- Configuring x64-linux-dbg
-- Configuring x64-linux-rel
-- Building x64-linux-dbg
-- Building x64-linux-rel
-- Installing: /content/vcpkg/packages/range-v3_x64-linux